# **Assignment 1**
The goal of the first assignment is to compute (in adjacency list format) the co-review graph of the Amazon Fine Food Reviews dataset, available at https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews.

### **1. Retrieve the dataset**

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [ ]:
!kaggle datasets download -d snap/amazon-fine-food-reviews

100% 242M/242M [00:08<00:00, 31.0MB/s]
100% 242M/242M [00:08<00:00, 29.4MB/s]


In [ ]:
!chmod 600 /content/kaggle.json

In [ ]:
!ls

amazon-fine-food-reviews.zip  kaggle.json  sample_data


In [ ]:
from zipfile import ZipFile
# Create a ZipFile Object and load sample.zip in it
with ZipFile('amazon-fine-food-reviews.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

In [ ]:
!ls

amazon-fine-food-reviews.zip  database.sqlite  hashes.txt  kaggle.json	Reviews.csv  sample_data


In [ ]:
!head -10 Reviews.csv

Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.
2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,"Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as ""Jumbo""."
3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all","This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally co

### Pre-processing on the original dataset

In [ ]:
import numpy as np
import pandas as pd
data = pd.read_csv('Reviews.csv', sep = ',')
df = data.replace([np.inf, -np.inf], np.nan) # Replacing infs with NaNs
df = data.dropna() # Dropping NaNs
df = df[df['HelpfulnessDenominator'] != float(0)]
df.describe()
df.to_csv('df.csv', index=False)

In [ ]:
# Given a Pandas frame raw_pd, save lines corresponding to approximately 10% of the users to file fout
# Note that all lines corresponding to sampled users are saved to fout

import pandas as pd
#import random

def sample_users(raw_pd, rate, buckets, fout):
    sampled_rows = []

    for index, row in raw_pd.iterrows():
        if (hash(row["UserId"]) % buckets < rate):
            sampled_rows.append(row)

    #create a dataframe of the sampled rows
    sampled_df = pd.DataFrame(sampled_rows)
    sampled_df.to_csv(fout, index=False)

sampled_file = "Reviews_sampled.csv"
rate = 10  # 10% of users
buckets = 100

fout = open(sampled_file, "w")
raw_pd = pd.read_csv('df.csv', sep = ',')
sample_users(raw_pd, rate, buckets, fout)
fout.close()

In [ ]:
sample = pd.read_csv('Reviews_sampled.csv', sep = ',')
# check if the number of sampled rows correspond to the 10% of the entire number of rows
print(len(raw_pd), len(sample))

298372 30054


In [ ]:
raw_pd.describe()

,Id,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time
count,298372.000000,298372.000000,298372.000000,298372.000000,2.983720e+05
mean,283375.663072,3.322152,4.244195,3.979492,1.283038e+09
std,163281.848521,10.288814,11.060079,1.461170,5.081528e+07
min,1.000000,0.000000,1.000000,1.000000,9.408096e+08
25%,142107.750000,1.000000,1.000000,3.000000,1.252886e+09
50%,283615.500000,1.000000,2.000000,5.000000,1.296518e+09
75%,424478.250000,3.000000,4.000000,5.000000,1.323389e+09
max,568453.000000,866.000000,923.000000,5.000000,1.351210e+09


In [ ]:
sample.describe()

,Id,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time
count,30054.000000,30054.000000,30054.000000,30054.000000,3.005400e+04
mean,282932.158082,3.217342,4.117888,4.009250,1.283302e+09
std,162566.596076,8.435763,9.210493,1.440179,5.056498e+07
min,15.000000,0.000000,1.000000,1.000000,9.657792e+08
25%,143163.750000,1.000000,1.000000,3.000000,1.253318e+09
50%,283289.500000,1.000000,2.000000,5.000000,1.296864e+09
75%,421910.500000,3.000000,4.000000,5.000000,1.323130e+09
max,568446.000000,580.000000,593.000000,5.000000,1.351210e+09


In [ ]:
sample.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,15,B001GVISJM,A2MUGFV2TDQ47K,"Lynrie ""Oh HELL no""",4,5,5,1268352000,Strawberry Twizzlers - Yummy,The Strawberry Twizzlers are my guilty pleasur...
1,33,B001EO5QW8,AOVROBZ8BNTP7,S. Potter,19,19,4,1163376000,Best of the Instant Oatmeals,McCann's Instant Oatmeal is great if you must ...
2,35,B001EO5QW8,A2EB6OGOWCRU5H,CorbyJames,9,9,5,1175212800,Great Irish oatmeal for those in a hurry!,Instant oatmeal can become soggy the minute th...
3,38,B001EO5QW8,A3MGP2E1ZZ6GRB,"Zardoz ""focuspuller""",1,1,5,1238457600,it's oatmeal,"What else do you need to know? Oatmeal, instan..."
4,52,B000G6RPMY,A3S5KJDA6ED2PS,Mike Kaser,4,4,5,1243900800,HOT! And good! Came back for more :),Got a free package of these with a bottle of b...


### **2. Use Apache pySpark to compute, for every user u, the list of all users who reviewed at least one item in common with u. For each such v, you should compute the number of items u and v co-reviewed.**

### **3. In more detail, you should return and store on a file a list of tuples with the following format:$[(<user>, <list>)]$**

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=4026c91fda43d479cd74fdbef259cabbd13dc3dae0593bbb768e152e7d363f05
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


### Find the co-review graph for the sampled reviews

In [ ]:
# Import needed packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, collect_list, struct

# SparkSession initialiation (to work with Dataframes)
spark = SparkSession.builder.appName("CoReviewGraph").getOrCreate()

# Read data
reviews_sampled = spark.read.csv("Reviews_sampled.csv", header=True, inferSchema=True)

# Select relevant columns and drop duplicates
user_product_pairs = reviews_sampled.select("UserId", "ProductId").distinct()

# Create alias user_product_pairs Dataframes
u = user_product_pairs.alias("u")
v = user_product_pairs.alias("v")

# Self join on the aliased Dataframes based on common ProductId but different UserId (to identify users who reviewd the same poduct)
co_review_pairs = u.join(v,(col("u.ProductId") == col("v.ProductId")) & (col("u.UserId") != col("v.UserId")),"inner")

# Count common reviews
co_review_count = co_review_pairs.groupBy("u.UserId", "v.UserId").count()
# Sort the list of tuples for each user in descending order (more common reviews will appear first)
co_review_count = co_review_count.orderBy(["u.UserId", "count"], ascending=[True, False])

# Group by user and aggregate the result by collecting a list of tuples containing the UserId and the count of common reviews
result = co_review_count.groupBy("u.UserId").agg(collect_list(struct(col("v.UserId"), col("count"))).alias("list"))
# Select the columns and aliases them as 'user_id' and 'list'
result_df = result.select(col("UserId").alias("user_id"), col("list"))

# Save data to a txt file
with open("co_review_graph.txt", "w") as file:
    # iterate over each row in the Dataframe
    for row in result_df.collect():
        user_id = row["user_id"]
        user_list = row["list"]
        # sort the list of tuples by count in descending order
        user_list = sorted(user_list, key=lambda x: x[1], reverse=True)
        user_list = [(v, count) for (v, count) in user_list]
        # write the result to the file
        file.write(f"({user_id}, {user_list})\n")

# Stop SparkSession
spark.stop()

### Since it works for a sample, find the co-review graph for the whole dataset

In [ ]:
spark = SparkSession.builder.appName("CoReviewGraphTotal").getOrCreate()
reviews_sampled = spark.read.csv("df.csv", header=True, inferSchema=True)

user_product_pairs = reviews_sampled.select("UserId", "ProductId").distinct()

u = user_product_pairs.alias("u")
v = user_product_pairs.alias("v")

co_review_pairs = u.join(v,(col("u.ProductId") == col("v.ProductId")) & (col("u.UserId") != col("v.UserId")),"inner")

co_review_count = co_review_pairs.groupBy("u.UserId", "v.UserId").count()
co_review_count = co_review_count.orderBy(["u.UserId", "count"], ascending=[True, False])

result = co_review_count.groupBy("u.UserId").agg(collect_list(struct(col("v.UserId"), col("count"))).alias("list"))
result_df = result.select(col("UserId").alias("user_id"), col("list"))

with open("co_review_graph_total.txt", "w") as file:
    for row in result_df.collect():
        user_id = row["user_id"]
        user_list = row["list"]
        user_list = sorted(user_list, key=lambda x: x[1], reverse=True)
        user_list = [(v, count) for (v, count) in user_list]
        file.write(f"({user_id}, {user_list})\n")

spark.stop()

### Show the first 10 lines of the file

In [ ]:
file_path = 'co_review_graph_total.txt'

with open(file_path, 'r') as file:
    for _ in range(10):
        line = file.readline()
        if not line:
            break  # Break if the end of the file is reached
        print(line.strip())

(#oc-R103C0QSV1DF5E, [('A20IQLFES5BDPG', 1), ('#oc-R2CIG5JYXHVPEE', 1), ('#oc-R2R9TFK97GIFOR', 1), ('#oc-R2XHEXATHR854G', 1), ('#oc-R2M3KDKLRT9W0L', 1), ('#oc-R1CRW5BOSI2J9P', 1), ('A11JHI75H8BQV6', 1), ('#oc-R2SMX6LG3FFJ32', 1), ('A1XWUR6X3V3OOS', 1), ('#oc-R1J5M6S2EUERD4', 1), ('#oc-R2PS459CVHZKJL', 1), ('#oc-R1VHP4GD3MVIGG', 1), ('#oc-R20T17BLTNKA3', 1), ('#oc-R152UR09M996EM', 1), ('#oc-R37BIITTCKZKLJ', 1), ('A1O55UH8MKGX13', 1), ('#oc-R1WT8QXGZCW9RU', 1), ('#oc-R3CW88JBWTEZ0I', 1), ('#oc-R3J6DE3FJLIVMZ', 1), ('#oc-R1KU5HKP3S3TXV', 1), ('#oc-R1TNRTD1MQTVYM', 1), ('#oc-R2RUAJOPVDG63Z', 1), ('#oc-R3L950A9DQXOGF', 1), ('#oc-R36D56XD4RTHQP', 1), ('#oc-R3IX025L68IP59', 1), ('#oc-R18XETRRZHQVPE', 1), ('#oc-R2XPA5PDTSKFER', 1), ('#oc-R2ARZA89FMQ7NU', 1), ('#oc-R2KCIJHXTXT0KU', 1), ('#oc-RNFCOA3SE6CUA', 1), ('#oc-R2I5AU09IL1R95', 1), ('#oc-R1PZMXFC48NDDQ', 1), ('#oc-R3378ONZT42ML5', 1), ('#oc-R92ECWR98ATCZ', 1), ('ADFTHRZRUYA8D', 1), ('#oc-R2MOR8M0C3KSK7', 1), ('#oc-RRG64YR1U9G1Z', 1), ('#o